In [4]:
from cpymad.madx import Madx
import numpy as np
import matplotlib.pyplot as plt
plt.ion()
get_ipython().run_line_magic('matplotlib', 'inline')
%config InlineBackend.figure_format = 'retina' # retina display
import matplotlib.patches as patches

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

import time
from ipywidgets import interactive
import ipywidgets as widgets
from scipy.interpolate import interp1d

In [5]:
def startMadx():
    madx = Madx(stdout=False)
    madx.input('''
    TITLE, ’CAS Third Exercise’; 

    circum=1000.0;
    ncell = 20; !Number of cells 
    lcell = circum/ncell;
    lq = 3.00; !Length of a quadrupole

    !element definitions;
    !define bending magnet as multipole 
    !we have 4 bending magnets per cell
    mb:multipole,knl={2.0*pi/(4*ncell)};

    angle1=0;
    angle2=0;
    angle3=0;
    angle4=0;
    anglep=0;
    anglek=0;

    !kicker definition
    hkick1: hkicker, kick:=angle1;
    hkick2: hkicker, kick:=angle2;
    hkick3: hkicker, kick:=angle3;
    hkick4: hkicker, kick:=angle4;
    fastkick: hkicker, kick:=anglep;
    kickerk: hkicker, kick:=anglek;


    !define quadrupoles as multipoles 
    qf: multipole,knl:={0,0.0098*lq+qtrim_f}; 
    qd: multipole,knl:={0,-0.0098*lq+qtrim_d};

    // define the sextupoles as multipole
    lsex = 0.00001; // dummy length, only used in the sequence;

    // ATTENTION: must use knl:= and NOT knl= to match later ! 
    msf: multipole, knl:={0,0,ksf};
    msd: multipole, knl:={0,0,ksd};

    !sequence declaration;
    cas3: sequence, refer=centre, l=circum;
       start_machine: marker, at = 0;
       n = 1;
       while (n < ncell+1) {
        qf: qf,   at=(n-1)*lcell;
        msf: msf, at=(n-1)*lcell + lsex/2.0;
        mb: mb,   at=(n-1)*lcell + 0.15*lcell;
        mb: mb,   at=(n-1)*lcell + 0.35*lcell;
        qd: qd,   at=(n-1)*lcell + 0.50*lcell;
        msd: msd, at=(n-1)*lcell + 0.50*lcell + lsex/2.0;
        mb: mb,   at=(n-1)*lcell + 0.65*lcell;
        mb: mb,   at=(n-1)*lcell + 0.85*lcell;
        at=(n-1)*lcell;
        n = n + 1;
    }
    end_machine: marker at=circum;
    endsequence;

    !define the beam and its properties
    Beam, particle = proton, sequence=cas3, energy = 20.0;

    use, sequence=cas3;

    !!!!! very interesting option !!!!!
    select,flag=interpolate, class=drift, slice=5, range=#s/#e;

    ksf=0;
    ksd=0;
    twiss;
    ''')
    return madx

def installKickers(madx,s0,d):
    madx.input(f"""
    seqedit, sequence=cas3;
        flatten;
        install, element=mark1, class=hkick1, at={s0-3*d/2};
        install, element=mark2, class=hkick2, at={s0-d/2};
        install, element=markp, class=fastkick, at={s0};
        install, element=mark3, class=hkick3, at={s0+d/2};
        install, element=mark4, class=hkick4, at={s0+3*d/2};
        flatten;
    endedit;

    use, sequence=cas3;
    """)

def calculateCorrectors(madx, xx,Ks, s0, d, installFlag=False):
    if installFlag:
        installKickers(madx,s0,d)

    madx.twiss()
    myTwiss=madx.table.twiss.dframe()

    aux1=myTwiss[myTwiss['name']=='mark1:1']
    mx1=aux1['mux'].values[0]*2*np.pi
    bx1=aux1['betx'].values[0]
    ax1=aux1['alfx'].values[0]
    #gx1=(1+ax1**2)/bx1

    aux2=myTwiss[myTwiss['name']=='mark2:1']
    mx2=aux2['mux'].values[0]*2*np.pi
    bx2=aux2['betx'].values[0]
    ax2=aux2['alfx'].values[0]
    #gx2=(1+ax2**2)/bx2

    aux3=myTwiss[myTwiss['name']=='mark3:1']
    mx3=aux3['mux'].values[0]*2*np.pi
    bx3=aux3['betx'].values[0]
    ax3=aux3['alfx'].values[0]
    #gx3=(1+ax3**2)/bx3

    aux4=myTwiss[myTwiss['name']=='mark4:1']
    mx4=aux4['mux'].values[0]*2*np.pi
    bx4=aux4['betx'].values[0]
    ax4=aux4['alfx'].values[0]
    #gx4=(1+ax4**2)/bx4

    auxp=myTwiss[myTwiss['name']=='markp:1']
    mxp=auxp['mux'].values[0]*2*np.pi
    bxp=auxp['betx'].values[0]
    axp=auxp['alfx'].values[0]
    xp = auxp['px'].values[0]
    
    
    #gxp=(1+axp**2)/bxp

    k1 = xx*(1/np.sqrt(bx1*bxp))*(np.cos(mxp-mx2)-axp*np.sin(mxp-mx2))/np.sin(mx2-mx1) - xp * np.sqrt(bxp/bx1) * (np.sin(mxp-mx2)/np.sin(mx2-mx1))
    k2 = xx*(-1/np.sqrt(bx2*bxp))*(np.cos(mxp-mx1)-axp*np.sin(mxp-mx1))/np.sin(mx2-mx1) + xp * np.sqrt(bxp/bx2) * (np.sin(mxp-mx1)/np.sin(mx2-mx1))

    k3 = xx*(-1/np.sqrt(bx3*bxp))*(np.cos(mx4-mxp)+axp*np.sin(mx4-mxp))/np.sin(mx4-mx3) - xp * np.sqrt(bxp/bx3) * (np.sin(mx4-mxp)/np.sin(mx4-mx3))
    k4 = xx*(1/np.sqrt(bx4*bxp))*(np.cos(mx3-mxp)+axp*np.sin(mx3-mxp))/np.sin(mx4-mx3) + xp * np.sqrt(bxp/bx4) * (np.sin(mx3-mxp)/np.sin(mx4-mx3))

    KsNew = np.zeros(4) 
    KsNew[0] = Ks[0] + k1
    KsNew[1] = Ks[1] + k2
    KsNew[2] = Ks[2] + k3
    KsNew[3] = Ks[3] + k4
    
    madx.input(f"""
    angle1 = {KsNew[0]};
    angle2 = {KsNew[1]};
    angle3 = {KsNew[2]};
    angle4 = {KsNew[3]};
    """)
    
    madx.twiss()
    myTwiss=madx.table.twiss.dframe()

    return KsNew, myTwiss

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx


def SolveCorrectionProblem(s0,d):
    madx = startMadx()
    KsNew, myTwiss = calculateCorrectors(madx, 0.002,np.zeros(4), s0, d, True)
    myTwiss0 = myTwiss.copy()

    for i in range(1):
        x=myTwiss[myTwiss['name']=='markp:1']['x'].values[0]
        KsNew, myTwiss = calculateCorrectors(madx, 0.002-(x),KsNew, s0, d, False)
    
    return madx, myTwiss0, myTwiss, KsNew

def find_s_from_phase(twiss,s0,phase):
    array1 = np.arange(s0-200,s0,step=0.01)
    s_full = np.append((twiss['s']+0.00001%1000)-1000,twiss['s'])
    mu_full = np.append((twiss['mux']- twiss['mux'][-1])*360 % 360,twiss['mux']*360 % 360)

    f = interp1d(s_full,mu_full)

    i =find_nearest(f(array1),(f(s0)-phase) %360)
    s_kick = array1[i]
    return s_kick % 1000


def calculateKicker(madx, xx, k1, s0, dk, installFlag=False):
    if installFlag:
        installFastKicker(madx,s0,dk)

    twiss_start = madx.table['twiss'].dframe()
    start = twiss_start.iloc[0]
    madx.twiss(x=0, px=0, betx=start['betx'], alfx=start['alfx'], bety=start['bety'], alfy=start['alfy'])
    myTwiss=madx.table.twiss.dframe()

    aux1=myTwiss[myTwiss['name']=='markk:1']
    mx1=aux1['mux'].values[0]*2*np.pi
    bx1=aux1['betx'].values[0]
    ax1=aux1['alfx'].values[0]
    #gx1=(1+ax1**2)/bx1


    auxp=myTwiss[myTwiss['name']=='markp:1']
    mxp=auxp['mux'].values[0]*2*np.pi
    bxp=auxp['betx'].values[0]
    axp=auxp['alfx'].values[0]
    xp = auxp['px'].values[0]
    
    
    #gxp=(1+axp**2)/bxp
    
    m12 =  np.sqrt(bx1*bxp)*np.sin(mxp-mx1)
    m22 = np.sqrt(bx1/bxp)*(axp*np.sin(mx1-mxp) + np.cos(mx1-mxp))

    k1New = k1 + xx/m12
    xp_septum = m22*k1New
    
    madx.input(f"""
    anglek = {k1New};
    """)
    
    madx.twiss(x=0, px=0, betx=start['betx'], alfx=start['alfx'], bety=start['bety'], alfy=start['alfy'])
    myTwiss=madx.table.twiss.dframe()

    return k1New, myTwiss

def installFastKicker(madx,s0,dk):
    madx.input(f"""
    seqedit, sequence=cas3;
        flatten;
        install, element=markk, class=kickerk, at={dk};
        flatten;
    endedit;

    use, sequence=cas3;
    """)
    
def SolveKickProblem(madx, s0, dk):
    K1New, myTwiss = calculateKicker(madx, 0.002,0, s0, dk, True)
    myTwiss0 = myTwiss.copy()

    for i in range(0):
        x=myTwiss[myTwiss['name']=='markp:1']['x'].values[0]
        K1New, myTwiss = calculateKicker(madx, 0.004-(x),K1New, s0, dk, False)
    
    return madx, myTwiss0, myTwiss, K1New


def find_s_from_phase(twiss,s0,phase):
    array1 = np.arange(s0-200,s0,step=0.01)
    s_full = np.append((twiss['s']+0.00001%1000)-1000,twiss['s'])
    mu_full = np.append((twiss['mux']- twiss['mux'][-1])*360 % 360,twiss['mux']*360 % 360)

    f = interp1d(s_full,mu_full)

    i =find_nearest(f(array1),(f(s0)-phase) %360)
    s_kick = array1[i]
    return s_kick % 1000



def plot_ellipse(beta, alpha, epsilon, ax, x0=0, px0=0, facecolor='black', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    
    gamma = (1+alpha**2)/beta
    cov = epsilon*np.array([[beta, -alpha], [-alpha, gamma]])
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensional dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the standard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0])
    mean_x = x0

    scale_y = np.sqrt(cov[1, 1])
    mean_y = px0

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

def Prepare_ellipse(twiss_final,ax):
    auxp=twiss_final[twiss_final['name']=='markp:1']
    mxp=auxp['mux'].values[0]*2*np.pi
    bxp=auxp['betx'].values[0]
    axp=auxp['alfx'].values[0]
    xp = auxp['px'].values[0]
    xt = auxp['x'].values[0]
    
    epsilon = 1e-7
    gamma= (1+axp**2)/bxp
    x_lim = np.sqrt(bxp*epsilon)
    y_lim = np.sqrt(gamma*epsilon)
    plot_ellipse(beta=bxp, alpha=axp, epsilon=epsilon, ax=ax, x0=xt, px0=xp)
    ax.set_xlim([-x_lim-xt,x_lim+xt])
    ax.set_ylim([-y_lim,y_lim]+xp)
    ax.grid()
    ax.scatter(xt, xp)
    ax.scatter(0,0)
    ax.set_title('Phase Space Ellips')
    ax.set_xlabel('x [m]')
    ax.set_ylabel(r"$x^'$ [m]")
    
def plotLatticeSeries(ax,s,l, height=1., v_offset=0., color='r',alpha=0.5,lw=3):
    ax.add_patch(
    patches.Rectangle(
        (s-l, v_offset-height/2.),   # (x,y)
        l,          # width
        height,          # height
        color=color, alpha=alpha,lw=lw
    )
    )
    return;
    
def plot_corrector_strengths(s0, d0, ax, KsNew):
    Ks = KsNew*1000
    locs = [s0 - (3*d0/2), s0 - (1*d0/2) , s0 + (1*d0/2), s0 + (3*d0/2)]
    #plotLatticeSeries(ax, s0 , d0/10, height=max(Ks) + max(Ks)/8, v_offset=d0/20, color='b')
    for i in range(len(KsNew)):
        plotLatticeSeries(ax, locs[i] , d0/8, height=Ks[i], v_offset=Ks[i]/2, color='r')
    color = 'r'
    ax.set_ylabel('angle [mm rad]', color=color)  # we already handled the x-label with ax1
    ax.tick_params(axis='y', labelcolor=color)
    ax.set_ylim([min(Ks) + min(Ks)/8, max(Ks) + max(Ks)/8])
    ax.set_xlabel(r's [m]')
    ax.set_xlim([s0 - 3*d0,s0 + 3*d0])
    ax.grid()
    ax.set_title('Corrector Strengths')
    
def PlotBetaBeating(s0,d0,ax, tw0, tw):
    btx0 = np.array(tw0['betx'].values)
    btx = np.array(tw['betx'].values)
    
    beating = (btx-btx0)/btx0
    
    ax.plot(tw['s'],beating,'g-')
    
    color = 'g'
    ax.set_ylabel(r'$\frac{\beta - \beta_0}{\beta_0}$', color=color)  # we already handled the x-label with ax1
    ax.tick_params(axis='y', labelcolor=color)
#     ax.set_ylim([min(Ks) + min(Ks)/8, max(Ks) + max(Ks)/8])
    ax.set_xlim([s0 - 3*d0,s0 + 3*d0])
    ax.grid()
    ax.set_title('Beta Beating')
    
   

In [6]:
 

def plotIt(s0,d0,KickerOn,phaseAdv):
    fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize=(20,12),gridspec_kw={'hspace':0.3})
    plt.subplots_adjust(left=0.25, bottom=0.25)
    try:
        madx, twiss0, twiss, KsNew = SolveCorrectionProblem(s0,d0)
        colors = 'b'

        if KickerOn:
            s_kick = find_s_from_phase(twiss,s0,phaseAdv)
            madx, twiss1, twiss_final, K1New = SolveKickProblem(madx, s0=s0, dk=s_kick)
        else:
            twiss_final = twiss.copy()
    except:
        madx, twiss0, twiss, KsNew = SolveCorrectionProblem(12,5)
        twiss_final = twiss.copy()
        colors = 'r'
        
    minIdx = find_nearest(twiss_final['s'].values, s0 - 5*d0)
    maxIdx = find_nearest(twiss_final['s'].values, s0 + 10*d0)

    t = np.array(twiss_final['s'].values)[minIdx:maxIdx]
    x = np.array(twiss_final['x'].values)[minIdx:maxIdx]

    l1, = ax1.plot(t, x*1000,colors, lw=2)
    ax1.set_title('Horizontal position')
    ax1.set_ylabel(r'x [mm]', color=colors)
    ax1.set_xlabel(r's [m]') 
    ax1.tick_params(axis='y', labelcolor=colors)
    ax1.margins(x=0)
    ax1.set_xlim([s0 - 3*d0,s0 + 3*d0])
#     ax5 = ax1.twinx()

    xp = np.array(twiss_final['px'].values)[minIdx:maxIdx]

    l2, = ax3.plot(t, xp*1000,colors, lw=2)
    ax3.margins(x=0)
    ax3.set_xlim([s0 - 3*d0,s0 + 3*d0])
    ax3.set_title('Horizontal momentum')
    ax3.set_ylabel(r"$x^'$ [mm]", color=colors) 
    ax3.set_xlabel(r's [m]')
    ax3.tick_params(axis='y', labelcolor=colors)
    
    Prepare_ellipse(twiss_final,ax4)
    plot_corrector_strengths(s0, d0, ax2, KsNew)
#     PlotBetaBeating(s0, d0, ax3, twiss0, twiss)
    
interactive_plot = interactive(plotIt,
                               s0=widgets.IntSlider(min=200, max=500, step=1, value=217),
                               d0=widgets.IntSlider(min=2, max=40, step=1, value=13),
                               KickerOn=[('False',False),('True',True)],
                               phaseAdv=widgets.IntSlider(min=10, max=170, step=5, value=90),
                               continuous_update=True)
output = interactive_plot.children[-1]
output.layout.height = '600px'
interactive_plot

interactive(children=(IntSlider(value=217, description='s0', max=500, min=200), IntSlider(value=13, descriptio…